## 0.PyTorch编程基础

**学习目标**

1. 熟练搭建PyTorch的软件开发环境
   
2. 了解PyTorch的张量计算
   
3. 了解PyTorch的自动求导机制
   
4. 了解PyTorch的数据集加载

****

PyTorch 是一个由 Facebook AI 研究团队开发的开源深度学习框架，广泛应用于学术研究和工业界。它以灵活性和动态计算图著称。PyTorch 使用动态计算图（称为 Autograd），允许在运行时构建和修改计算图，便于调试和灵活调整模型结构。PyTorch的动态计算图和 Python 优先设计使其成为深度学习领域的首选工具之一。同时该框架也提供 TorchVision、TorchText、TorchAudio 等库，支持计算机视觉、自然语言处理和音频处理等任务。PyTorch 的核心数据结构为张量（Tensor），类似于 NumPy 的 ndarray，但支持 GPU 加速和自动求导。

<img src="./images/PyTorch_logo.jpg" style="zoom:60%;" />

## 0.1 软件环境的安装

在我们下载并安装好Anaconda之后，打开Anaconda的虚拟控制台，然后创建一个新的环境，输入以下命令：

```
conda create --name dl2llm python=3.10
```

然后激活刚刚创建的环境：

```
conda activate dl2llm
```

接下来，我们需要安装pytorch。如果你的电脑带有NVIDIA显卡，保证显卡驱动版本是最新，然后在命令行中输入以下命令：

```
pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121
```


如果你的电脑没有NVIDIA显卡，则输入以下命令：

```
pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 -i https://pypi.tuna.tsinghua.edu.cn/simple
```

在安装过程出现问题，请参考[PyTorch官网](https://pytorch.org/get-started/locally/)的安装说明。

最后安装课程中的其他相关依赖包：

```
pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple
```

## 0.2 PyTorch的核心数据结构——张量（Tensor）

Tensors 是一种特殊的数据结构，与数组和矩阵非常相似。在 PyTorch 中，我们使用 tensors 对模型的输入和输出以及模型的参数进行编码。

Tensors 类似于 NumPy’s 的 ndarrays，不同的是 tensors 可以在 GPU 或其他硬件加速器上运行。事实上，tensors 和 NumPy 数组通常可以共享相同的底层内存，这样就不需要复制数据了(参见使用 NumPy 的 Bridge)。 Tensors 也针对自动微分进行了优化(我们将在后面的 Autograd 部分中看到更多细节介绍)。如果您熟悉 ndarray，那么您就会熟悉 Tensor API。

1. 创建张量

In [2]:
import torch

# 从 Python 列表创建
x = torch.tensor([1, 2, 3])
# 创建全零张量
zeros = torch.zeros(2, 3)  # 2行3列的全零矩阵
# 创建全一张量
ones = torch.ones(2, 3)    # 2行3列的全一矩阵
# 创建随机张量
rand_tensor = torch.rand(2, 3)  # 2行3列的随机值矩阵

print(x)
print(zeros)
print(ones)
print(rand_tensor)

tensor([1, 2, 3])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.1313, 0.6651, 0.3656],
        [0.5114, 0.5825, 0.2803]])


In [4]:
# 从 NumPy 数组创建
import numpy as np
numpy_array = np.array([1, 2, 3])
tensor_from_numpy = torch.from_numpy(numpy_array)
print(tensor_from_numpy)

tensor([1, 2, 3], dtype=torch.int32)


2. Tensor 操作

tensor 有超过100种操作，包括算术、线性代数、矩阵操作(转置、索引、切片) 、抽样等。这些操作都可以在 GPU 上运行(通常比在 CPU 上运行的速度更快)。默认情况下，tensors 是在 CPU 上创建的。我们需要使用 .to(device) 方法显式地将 tensors 移动到 GPU 上(在检查GPU的可用性之后)。请记住，在不同的设备上复制大型的 tensors，在时间和内存上都是很昂贵的!

In [5]:
# 加法
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])
c = a + b  # 或者 torch.add(a, b)
print(c)  # tensor([5, 7, 9])

tensor([5, 7, 9])


In [6]:
# 矩阵乘法
mat1 = torch.rand(2, 3)
mat2 = torch.rand(3, 2)
result = torch.matmul(mat1, mat2)  # 或者 mat1 @ mat2
print(result)

tensor([[0.8976, 1.2730],
        [1.0100, 1.4954]])


In [7]:
# 改变形状
x = torch.rand(2, 3)
reshaped = x.view(3, 2)  # 改变为 3行2列
print(reshaped)

tensor([[0.4493, 0.0624],
        [0.4306, 0.6429],
        [0.5024, 0.6473]])


3. 与NumPy的关系

CPU 上的 Tensors 和 NumPy 数组可以共享它们的底层内存存储，更改其中一个将更改另一个。这意味着你可以在 PyTorch 中使用 NumPy 数组，反之亦然。

Tensor转为NumPy数组

In [10]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


NumPy数组转为Tensor

In [11]:
n = np.ones(5)
t = torch.from_numpy(n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
n: [1. 1. 1. 1. 1.]


## 0.3 PyTorch的自动求导机制 

PyTorch 的 autograd 模块支持自动求导，是训练神经网络的核心。

In [17]:
# 创建一个需要梯度的张量
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# 定义一个计算
y = x ** 2
z = y.sum() # 计算和

# 反向传播计算梯度
z.backward()

# 查看梯度
print(x.grad)  # tensor([2., 4., 6.])
print(y)
print(z)

tensor([2., 4., 6.])
tensor([1., 4., 9.], grad_fn=<PowBackward0>)
tensor(14., grad_fn=<SumBackward0>)


## 0.4  PyTorch的数据集加载

PyTorch有两个处理数据的基本操作：torch.utils.data.DataLoader和torch.utils.data.Dataset。Dataset用于存储样本及其对应的标签，而DataLoader则围绕Dataset包装了一个可迭代的数据加载器。

Dataset 类需要继承 torch.utils.data.Dataset 类，并实现两个方法：

- `__len__`：返回数据集的长度。
- `__getitem__`：返回数据集中第 index 项的数据。

DataLoader 类需要继承 torch.utils.data.DataLoader 类，并实现两个方法：

- `__iter__`：返回一个迭代器，用于按顺序访问数据集中的每一项。
- `__len__`：返回数据集的长度。

In [9]:
# 自定义 Dataset
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 示例数据
data = torch.tensor([[1.0], [2.0], [3.0]])
labels = torch.tensor([[2.0], [4.0], [6.0]])
dataset = CustomDataset(data, labels)

# 使用 DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
print(next(iter(dataloader)))

[tensor([[2.],
        [1.]]), tensor([[4.],
        [2.]])]


PyTorch 提供特定于领域的库，如 TorchText, TorchVision 和 TorchAudio，所有这些库都包含数据集。这里我们使用 TorchVision 数据集。torchvision.datasets模块包含了许多真实世界视觉数据的 Dataset对象，比如 CIFAR、 COCO 等。每个 TorchVision Dataset都包含两个参数: transform 和 target_transform，分别用于转换样本和标签。

我们使用 FashionMNIST 数据集为例来说明如何使用。FashionMNIST 数据集是一个 28x28 的灰度图像分类数据集，共 60,000 张训练图像和 10,000 张测试图像。每张图像都是一个 28x28 的灰度图像，其标签是 10 种服饰的类别之一。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# 从开源数据集下载训练数据。
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 从开源数据集下载测试数据。
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)